# MTA BusTime SIRI API demo

## setup

In [1]:
!pip install requests trio asks

     |████████████████████████████████| 356 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 537 kB/s  eta 0:00:01
  Created wheel for asks: filename=asks-2.4.12-py3-none-any.whl size=25680 sha256=a948c9416a8072dd5507209bdc581ce6e0b8e7137b70229b15001b94c90b9832
  Stored in directory: /Users/anthonytownsend/Library/Caches/pip/wheels/5c/37/5a/435ec69c044db2e04ac60f3cec7874d7011966e66e887ba15f
Successfully built asks


In [ ]:
import requests

### create a timer decorator (from RealPython)

In [ ]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

### request url generators

In [ ]:
API_KEY = '088886bd-cc48-4d7c-bd8a-498d353d7d13'

def url_fetch_all()):
    return f'http://bustime.mta.info/api/siri/vehicle-monitoring.json?key={API_KEY}&VehicleMonitoringDetailLevel=calls'

def url_fetch_1(route):
    return f'http://bustime.mta.info/api/siri/vehicle-monitoring.json?key={API_KEY}&VehicleMonitoringDetailLevel=calls&LineRef={route}'


# fetch entire system

In [ ]:
@timer
def get_all():
    response = requests.get(url_fetch_all)
    jsonResponse = response.json()
    return jsonResponse

buses = get_all()['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
len(buses)

# fetch a single route

In [ ]:
@ timer
def get_one(route):
    response = requests.get(url_fetch_1(route))
    jsonResponse = response.json()
    return jsonResponse

buses = get_one('M15')['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
len(buses)

# fetch entire system one route at a time

## get the list of routes from a sister API, MTA BusTime's One Bus Away

In [ ]:
def get_OBA_routelist():
    url = "http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json?key=" + API_KEY
    response = requests.get(url, timeout=30)
    return [x['id'] for x in response.json()['data']['list']]

print ( get_OBA_routelist() )


## iterate over the list, calling get_one() for each route

In [ ]:
def get_one(route): #no timer decorator
    response = requests.get(url_fetch_1(route))
    jsonResponse = response.json()
    return jsonResponse

@ timer
def get_all_one_at_a_time():
    responses = []
    for route in get_OBA_routelist():
        
        jsonResponse = get_one(route)
        try:
            print(
                f"{len(jsonResponse['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'])} buses on {route}"
            )
        except:
            print(f'0 buses on {route}')
        responses.append(jsonResponse)
    return responses

responses = get_all_one_at_a_time()


# fetch entire system one route at a time — asynchronously...

In [10]:
API_KEY = '088886bd-cc48-4d7c-bd8a-498d353d7d13'

import requests
from time import time
import datetime as dt
import trio

def async_grab_and_store(localhost, cwd):
    start = time()
    SIRI_request_urlpaths = get_SIRI_request_urlpaths()
    feeds = []

    async def grabber(s,a_path,route_id):
        try:
            r = await s.get(path=a_path, retries=10)
            feeds.append({route_id:r})
        except Exception as e:
            logging.error ('\tCould not fetch feed for {}. (Increase max retries for Session.get()?)'.format(route_id) )

    async def main(path_list):
        from asks.sessions import Session

        if localhost is True:
            s = Session('http://bustime.mta.info', connections=5)
        else:
            s = Session('http://bustime.mta.info', connections=config.config['http_connections'])
        async with trio.open_nursery() as n:
            for path_bundle in path_list:
                for route_id,path in path_bundle.items():
                    n.start_soon(grabber, s, path, route_id )

    trio.run(main, SIRI_request_urlpaths)

    # dump to the various locations
    timestamp = dt.datetime.now()
    # DataLake(cwd).make_puddles(feeds, DatePointer(timestamp))
    # DataStore(cwd).make_barrels(feeds, DatePointer(timestamp))

    # report results to console
    n_buses = num_buses(feeds)
    end = time()
    print('Grabbed {} route feeds containing {} buses in {:2f} seconds at {}.'.format(len(feeds),n_buses,(end - start), dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    return


def get_OBA_routelist():
    url = "http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json?key=" + API_KEY
    response = requests.get(url, timeout=30)
    routes = response.json()
    return routes


def get_SIRI_request_urlpaths():
    SIRI_request_urlpaths = []
    routes=get_OBA_routelist()
    for route in routes['data']['list']:
        SIRI_request_urlpaths.append({route['id']:"/api/siri/vehicle-monitoring.json?key={}&VehicleMonitoringDetailLevel=calls&LineRef={}".format(API_KEY, route['id'])})
    return SIRI_request_urlpaths


def num_buses(feeds):
    num_buses=0
    for route_report in feeds:
        for route_id,route_data in route_report.items():
            try:
                route_data = route_data.json()
                for monitored_vehicle_journey in route_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']:
                    num_buses = num_buses + 1
            except: # no vehicle activity?
                pass
    return num_buses


In [11]:
localhost = True
cwd = '.'
async_grab_and_store(localhost, cwd)

Grabbed 241 route feeds containing 579 buses in 4.958737 seconds at 2021-08-18 23:56:17.
